# Programación a BD

In [1]:
import pandas as pd
from utils.aggregation_functions import (rellenar_etas, agrupar_descargas,
                                         estimar_demurrage, formato_BD)
from utils.extraction_functions import (extraer_bts, extraer_descargas, extraer_tiempos_de_viaje,
                   extraer_planificacion, extraer_programas, extraer_nueva_ficha, extraer_productos_plantas,
                   extraer_reporte_tankers)

In [2]:
PATH_DISTANCIAS = "Distancias entre puertos.xlsx"
PATH_PROGRAMACION = "C:\\Users\\jaubele.ap\\Desktop\\Estimación Semanal\\Programaciones\\Programacion Descarga Importaciones 03 de NOV.xlsx"
PATH_NUEVA_FICHA = "Nueva Ficha Información de Buques.xlsx"
PATH_REPORTES_TANKERS = "C:\\Users\\jaubele.ap\\Desktop\\Estimación Semanal\\Reportes Tankers\\Reporte tankers 04-nov-25.pdf"
FECHA_PROGRAMACION = pd.to_datetime("2025-11-04")
FILE_NAME = f"Base de datos Estimación Semanal {FECHA_PROGRAMACION.strftime('%d-%m-%Y')}.xlsx"

In [3]:
df_bts = extraer_bts(PATH_PROGRAMACION, "Buques")
df_planificacion = extraer_planificacion(PATH_PROGRAMACION, "Planificación")
df_descargas = extraer_descargas(df_planificacion, ignore_not_bts=True, df_bts=df_bts)
df_programas = extraer_programas(df_planificacion)
df_productos_plantas = extraer_productos_plantas()
df_nueva_ficha = extraer_nueva_ficha(PATH_NUEVA_FICHA, "Programación de buques", df_programas=df_programas)
df_reporte_tankers = extraer_reporte_tankers(PATH_REPORTES_TANKERS)
matriz_de_tiempos = extraer_tiempos_de_viaje("Distancias entre puertos.xlsx", "Datos")

df_descargas_productos_plantas = df_descargas.merge(df_productos_plantas, on=["Columna"]).drop(columns=["Columna"])
df_descargas_completo = df_descargas_productos_plantas.merge(df_bts, on=["Abrev."]).drop(columns=["Abrev."])
df_descargas_completo = df_descargas_completo[["Fecha", "N° Referencia", "Nombre programa", "Nombre del BT",
                                            "Producto", "Planta", "Ciudad", "Alias", "Volumen"]]

df_descargas_agrupadas = agrupar_descargas(df_descargas_completo)
df_descargas_agrupadas

Exception: ('Abreviaturas duplicadas encontradas en la hoja "Buques".',    N° Referencia       Nombre programa  Nombre del BT Abrev.
8      CC 120/25  120/25 Silver Esther  Silver Esther     SE
10      CC 54/25      54/25 STI Mighty     STI Mighty     SE)

In [ ]:
df_bts[df_bts["Abrev."].duplicated(keep=False)]

,N° Referencia,Nombre programa,Nombre del BT,Abrev.


In [ ]:
# Selecciona de la nueva ficha solo los programas de la programación
df_programas_completo = df_programas.merge(df_nueva_ficha, on="N° Referencia", how="left")

# Considera preferentemente las ventanas cortas, si no existen, las ventanas largas
df_programas_completo["Inicio Ventana"] = df_programas_completo["Inicio Ventana Corta"].combine_first(df_programas_completo["Inicio Ventana"])
df_programas_completo["Fin Ventana"] = df_programas_completo["Fin Ventana Corta"].combine_first(df_programas_completo["Fin Ventana"])

# Considera la ETA de la nueva ficha, si no existe, la ETA del programa.
df_programas_completo["ETA"] = df_programas_completo["ETA"].combine_first(df_programas_completo["ETA Programa"])

# Rellena los demurrages rates faltantes con la media
df_programas_completo["MONTO ($/DIA)"] = df_programas_completo["MONTO ($/DIA)"].fillna(df_programas_completo["MONTO ($/DIA)"].mean()).astype(int)

df_programas_completo = df_programas_completo.drop(columns=["Inicio Ventana Corta", "Fin Ventana Corta", "ETA Programa"])
df_programas_completo.index = range(1, len(df_programas_completo) + 1)
df_programas_completo

,N° Referencia,Proveedor,Inicio Ventana,Fin Ventana,ETA,MONTO ($/DIA)
1,CC 138/25,ARAMCO,2025-11-03,2025-11-07 23:59:59,2025-11-03,34000
2,CC 53/25,CHEVRON,2025-11-03,2025-11-07 23:59:59,2025-11-04,38000
3,CC 120/25,BB ENERGY,2025-11-03,2025-11-07 23:59:59,2025-11-05,34500
4,CC 54/25,CHEVRON,2025-11-10,2025-11-14 23:59:59,2025-11-10,34500
5,CC 131/25,ARAMCO,2025-11-10,2025-11-14 23:59:59,2025-11-14,36000
6,CC 23/25,ARAMCO,2025-11-17,2025-11-21 23:59:59,2025-11-17,33500
7,CC 136/25,PETROCHINA,2025-11-17,2025-11-21 23:59:59,2025-11-21,34500
8,CC 104/25,VITOL,2025-11-24,2025-11-28 23:59:59,2025-11-23,34500
9,CC 91/25,CHEVRON,2025-11-21,2025-11-25 23:59:59,2025-11-24,34500
10,CC 55/25,CHEVRON,2025-11-24,2025-11-28 23:59:59,2025-11-25,34500


In [ ]:
# Actualiza la información de ventanas y ETA con el reporte tankers si existe
df_programas_completo = df_programas_completo.merge(df_reporte_tankers, on=["N° Referencia"], how="left", suffixes=("", " Reporte Tankers"))
df_programas_completo["Inicio Ventana"] = df_programas_completo["Inicio Ventana Reporte Tankers"].combine_first(df_programas_completo["Inicio Ventana"])
df_programas_completo["Fin Ventana"] = df_programas_completo["Fin Ventana Reporte Tankers"].combine_first(df_programas_completo["Fin Ventana"])
df_programas_completo["ETA"] = df_programas_completo["ETA Reporte Tankers"].combine_first(df_programas_completo["ETA"])
df_programas_completo = df_programas_completo.drop(columns=["Inicio Ventana Reporte Tankers", "Fin Ventana Reporte Tankers", "ETA Reporte Tankers"])
df_programas_completo

,N° Referencia,Proveedor,Inicio Ventana,Fin Ventana,ETA,MONTO ($/DIA)
0,CC 138/25,ARAMCO,2025-11-03,2025-11-05 23:59:59,2025-11-02 02:00:00,34000
1,CC 53/25,CHEVRON,2025-11-03,2025-11-05 23:59:59,2025-11-02 00:00:00,38000
2,CC 120/25,BB ENERGY,2025-11-05,2025-11-07 23:59:59,2025-11-03 00:00:00,34500
3,CC 54/25,CHEVRON,2025-11-10,2025-11-12 23:59:59,2025-11-08 00:00:00,34500
4,CC 131/25,ARAMCO,2025-11-12,2025-11-14 23:59:59,2025-11-15 00:00:00,36000
5,CC 23/25,ARAMCO,2025-11-17,2025-11-21 23:59:59,2025-11-17 20:00:00,33500
6,CC 136/25,PETROCHINA,2025-11-17,2025-11-21 23:59:59,2025-11-20 09:00:00,34500
7,CC 104/25,VITOL,2025-11-24,2025-11-28 23:59:59,2025-11-23 00:00:00,34500
8,CC 91/25,CHEVRON,2025-11-24,2025-11-28 23:59:59,2025-11-24 00:00:00,34500
9,CC 55/25,CHEVRON,2025-11-24,2025-11-28 23:59:59,2025-11-25 00:00:00,34500


In [ ]:
df_descargas_por_programa = df_descargas_agrupadas.merge(df_programas_completo, on="N° Referencia", how="right")
df_descargas_por_programa = df_descargas_por_programa[df_descargas_por_programa["Producto"].notna()]
df_descargas_por_programa

,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga,Proveedor,Inicio Ventana,Fin Ventana,ETA,MONTO ($/DIA)
0,138/25 Hellas Marianna,CC 138/25,Hellas Marianna,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-11-11 15:00:00,2025-11-12 23:00:00,10000,1,ARAMCO,2025-11-03,2025-11-05 23:59:59,2025-11-02 02:00:00,34000
1,138/25 Hellas Marianna,CC 138/25,Hellas Marianna,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-11-15 15:00:00,2025-11-16 23:00:00,38000,2,ARAMCO,2025-11-03,2025-11-05 23:59:59,2025-11-02 02:00:00,34000
2,53/25 Weco Memphis Belle,CC 53/25,Weco Memphis Belle,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-11-13 15:00:00,2025-11-14 23:00:00,10000,1,CHEVRON,2025-11-03,2025-11-05 23:59:59,2025-11-02 00:00:00,38000
3,53/25 Weco Memphis Belle,CC 53/25,Weco Memphis Belle,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-11-18 15:00:00,2025-11-19 23:00:00,38000,2,CHEVRON,2025-11-03,2025-11-05 23:59:59,2025-11-02 00:00:00,38000
4,120/25 Silver Esther,CC 120/25,Silver Esther,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-11-05 15:00:00,2025-11-06 23:00:00,12000,1,BB ENERGY,2025-11-05,2025-11-07 23:59:59,2025-11-03 00:00:00,34500
5,120/25 Silver Esther,CC 120/25,Silver Esther,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-11-13 15:00:00,2025-11-14 23:00:00,36000,2,BB ENERGY,2025-11-05,2025-11-07 23:59:59,2025-11-03 00:00:00,34500
6,54/25 STI Mighty,CC 54/25,STI Mighty,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-11-18 15:00:00,2025-11-19 23:00:00,10000,1,CHEVRON,2025-11-10,2025-11-12 23:59:59,2025-11-08 00:00:00,34500
7,54/25 STI Mighty,CC 54/25,STI Mighty,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-11-21 15:00:00,2025-11-22 23:00:00,38000,2,CHEVRON,2025-11-10,2025-11-12 23:59:59,2025-11-08 00:00:00,34500
8,131/25 Dat Venus,CC 131/25,Dat Venus,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-12-11 15:00:00,2025-12-12 23:00:00,38000,1,ARAMCO,2025-11-12,2025-11-14 23:59:59,2025-11-15 00:00:00,36000
9,131/25 Dat Venus,CC 131/25,Dat Venus,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-12-14 15:00:00,2025-12-15 23:00:00,10000,2,ARAMCO,2025-11-12,2025-11-14 23:59:59,2025-11-15 00:00:00,36000


In [ ]:
df_descargas_por_programa["ETA"] = df_descargas_por_programa["ETA"][[True if descarga == 1 else False for descarga in df_descargas_por_programa["N° Descarga"]]]
df_descargas_por_programa = rellenar_etas(df_descargas_por_programa, matriz_de_tiempos)
df_descargas_por_programa

,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga,Proveedor,Inicio Ventana,Fin Ventana,ETA,MONTO ($/DIA),NOR + 6,Shifting
0,138/25 Hellas Marianna,CC 138/25,Hellas Marianna,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-11-11 15:00:00,2025-11-12 23:00:00,10000,1,ARAMCO,2025-11-03,2025-11-05 23:59:59,2025-11-02 02:00:00,34000,True,NaN
1,138/25 Hellas Marianna,CC 138/25,Hellas Marianna,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-11-15 15:00:00,2025-11-16 23:00:00,38000,2,ARAMCO,2025-11-03,2025-11-05 23:59:59,2025-11-15 15:00:00,34000,True,NaN
2,53/25 Weco Memphis Belle,CC 53/25,Weco Memphis Belle,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-11-13 15:00:00,2025-11-14 23:00:00,10000,1,CHEVRON,2025-11-03,2025-11-05 23:59:59,2025-11-02 00:00:00,38000,True,NaN
3,53/25 Weco Memphis Belle,CC 53/25,Weco Memphis Belle,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-11-18 15:00:00,2025-11-19 23:00:00,38000,2,CHEVRON,2025-11-03,2025-11-05 23:59:59,2025-11-15 21:00:00,38000,True,NaN
4,120/25 Silver Esther,CC 120/25,Silver Esther,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-11-05 15:00:00,2025-11-06 23:00:00,12000,1,BB ENERGY,2025-11-05,2025-11-07 23:59:59,2025-11-03 00:00:00,34500,True,NaN
5,120/25 Silver Esther,CC 120/25,Silver Esther,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-11-13 15:00:00,2025-11-14 23:00:00,36000,2,BB ENERGY,2025-11-05,2025-11-07 23:59:59,2025-11-07 21:00:00,34500,True,NaN
6,54/25 STI Mighty,CC 54/25,STI Mighty,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-11-18 15:00:00,2025-11-19 23:00:00,10000,1,CHEVRON,2025-11-10,2025-11-12 23:59:59,2025-11-08 00:00:00,34500,True,NaN
7,54/25 STI Mighty,CC 54/25,STI Mighty,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-11-21 15:00:00,2025-11-22 23:00:00,38000,2,CHEVRON,2025-11-10,2025-11-12 23:59:59,2025-11-21 06:00:00,34500,True,NaN
8,131/25 Dat Venus,CC 131/25,Dat Venus,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-12-11 15:00:00,2025-12-12 23:00:00,38000,1,ARAMCO,2025-11-12,2025-11-14 23:59:59,2025-11-15 00:00:00,36000,True,NaN
9,131/25 Dat Venus,CC 131/25,Dat Venus,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-12-14 15:00:00,2025-12-15 23:00:00,10000,2,ARAMCO,2025-11-12,2025-11-14 23:59:59,2025-12-15 15:00:00,36000,True,NaN


## Parte 5: Calculo Demurrage/Laytime

In [ ]:
df_estimacion = estimar_demurrage(df_descargas_por_programa)
df_estimacion[["Arribo", "Nombre programa", "N° Descarga", "Planta", "Fecha inicio", "Fecha fin",
               "Inicio Ventana", "Fin Ventana", "ETA", "MONTO ($/DIA)", "Inicio Laytime", 
               "Laytime descarga (Horas)", "Laytime programa (Horas)", "Laytime pactado (Horas)",
                "Demurrage descarga (Horas)", "Demurrage programa (Horas)",
                "Estimación demurrage", "Demurrage unitario", "Shifting"]]

,Arribo,Nombre programa,N° Descarga,Planta,Fecha inicio,Fecha fin,Inicio Ventana,Fin Ventana,ETA,MONTO ($/DIA),Inicio Laytime,Laytime descarga (Horas),Laytime programa (Horas),Laytime pactado (Horas),Demurrage descarga (Horas),Demurrage programa (Horas),Estimación demurrage,Demurrage unitario,Shifting
0,Antes,138/25 Hellas Marianna,1,PLANTA IQUIQUE,2025-11-11 15:00:00,2025-11-12 23:00:00,2025-11-03,2025-11-05 23:59:59,2025-11-02 02:00:00,34000,2025-11-03 06:00:00,233.0,265.0,132,116.939623,133.0,165665.0,16.566500,NaN
1,Antes,138/25 Hellas Marianna,2,TERMINAL TPI,2025-11-15 15:00:00,2025-11-16 23:00:00,2025-11-03,2025-11-05 23:59:59,2025-11-15 15:00:00,34000,2025-11-15 15:00:00,32.0,265.0,132,16.060377,133.0,22753.0,0.598763,NaN
2,Antes,53/25 Weco Memphis Belle,1,PLANTA CALDERA,2025-11-13 15:00:00,2025-11-14 23:00:00,2025-11-03,2025-11-05 23:59:59,2025-11-02 00:00:00,38000,2025-11-03 06:00:00,281.0,373.0,132,181.557641,241.0,287467.0,28.746700,NaN
3,Antes,53/25 Weco Memphis Belle,2,PLANTA MEJILLONES,2025-11-18 15:00:00,2025-11-19 23:00:00,2025-11-03,2025-11-05 23:59:59,2025-11-15 21:00:00,38000,2025-11-16 03:00:00,92.0,373.0,132,59.442359,241.0,94118.0,2.476789,NaN
4,Antes,120/25 Silver Esther,1,PLANTA CALDERA,2025-11-05 15:00:00,2025-11-06 23:00:00,2025-11-05,2025-11-07 23:59:59,2025-11-03 00:00:00,34500,2025-11-05 06:00:00,41.0,205.0,132,14.600000,73.0,20988.0,1.749000,NaN
5,Antes,120/25 Silver Esther,2,PLANTA MEJILLONES,2025-11-13 15:00:00,2025-11-14 23:00:00,2025-11-05,2025-11-07 23:59:59,2025-11-07 21:00:00,34500,2025-11-08 03:00:00,164.0,205.0,132,58.400000,73.0,83950.0,2.331944,NaN
6,Antes,54/25 STI Mighty,1,PLANTA CALDERA,2025-11-18 15:00:00,2025-11-19 23:00:00,2025-11-10,2025-11-12 23:59:59,2025-11-08 00:00:00,34500,2025-11-10 06:00:00,233.0,268.0,132,118.238806,136.0,169969.0,16.996900,NaN
7,Antes,54/25 STI Mighty,2,TERMINAL TPI,2025-11-21 15:00:00,2025-11-22 23:00:00,2025-11-10,2025-11-12 23:59:59,2025-11-21 06:00:00,34500,2025-11-21 12:00:00,35.0,268.0,132,17.761194,136.0,25532.0,0.671895,NaN
8,Después,131/25 Dat Venus,1,TERMINAL TPI,2025-12-11 15:00:00,2025-12-12 23:00:00,2025-11-12,2025-11-14 23:59:59,2025-11-15 00:00:00,36000,2025-12-11 15:00:00,32.0,64.0,132,0.000000,0.0,0.0,0.000000,NaN
9,Después,131/25 Dat Venus,2,PLANTA IQUIQUE,2025-12-14 15:00:00,2025-12-15 23:00:00,2025-11-12,2025-11-14 23:59:59,2025-12-15 15:00:00,36000,2025-12-14 15:00:00,32.0,64.0,132,0.000000,0.0,0.0,0.000000,NaN


In [ ]:
df_BD = formato_BD(df_estimacion, df_descargas_completo, FECHA_PROGRAMACION)
df_BD

,Fecha de programación,Semana,Año,Mes,Horas Laytime,CC,Nombre BT,Proveedor,Producto,Demurrage,Puerto,Volumen,Inicio Ventana,Final Ventana,ETA,Fin descarga,Estimación demurrage,Demurrage unitario,Shifting
0,04-11-2025,Semana 2,2025,Noviembre,132,CC 138/25,Hellas Marianna,ARAMCO,Diesel A1,34000,Iquique,10000,03-11-2025,05-11-2025,02-11-2025 02:00,12-11-2025,165665.0,16.566500,NaN
1,04-11-2025,Semana 2,2025,Noviembre,132,CC 138/25,Hellas Marianna,ARAMCO,Diesel A1,34000,TPI,38000,03-11-2025,05-11-2025,15-11-2025 15:00,16-11-2025,22753.0,0.598763,NaN
2,04-11-2025,Semana 2,2025,Noviembre,132,CC 53/25,Weco Memphis Belle,CHEVRON,Diesel A1,38000,Caldera,10000,03-11-2025,05-11-2025,02-11-2025 00:00,14-11-2025,287467.0,28.746700,NaN
3,04-11-2025,Semana 2,2025,Noviembre,132,CC 53/25,Weco Memphis Belle,CHEVRON,Diesel A1,38000,Mejillones,38000,03-11-2025,05-11-2025,15-11-2025 21:00,19-11-2025,94118.0,2.476789,NaN
4,04-11-2025,Semana 2,2025,Noviembre,132,CC 120/25,Silver Esther,BB ENERGY,Diesel A1,34500,Caldera,12000,05-11-2025,07-11-2025,03-11-2025 00:00,06-11-2025,20988.0,1.749000,NaN
5,04-11-2025,Semana 2,2025,Noviembre,132,CC 120/25,Silver Esther,BB ENERGY,Diesel A1,34500,Mejillones,36000,05-11-2025,07-11-2025,07-11-2025 21:00,14-11-2025,83950.0,2.331944,NaN
6,04-11-2025,Semana 2,2025,Noviembre,132,CC 54/25,STI Mighty,CHEVRON,Diesel A1,34500,Caldera,10000,10-11-2025,12-11-2025,08-11-2025 00:00,19-11-2025,169969.0,16.996900,NaN
7,04-11-2025,Semana 2,2025,Noviembre,132,CC 54/25,STI Mighty,CHEVRON,Diesel A1,34500,TPI,38000,10-11-2025,12-11-2025,21-11-2025 06:00,22-11-2025,25532.0,0.671895,NaN
8,04-11-2025,Semana 2,2025,Diciembre,132,CC 131/25,Dat Venus,ARAMCO,Diesel A1,36000,TPI,38000,12-11-2025,14-11-2025,15-11-2025 00:00,12-12-2025,0.0,0.000000,NaN
9,04-11-2025,Semana 2,2025,Diciembre,132,CC 131/25,Dat Venus,ARAMCO,Diesel A1,36000,Iquique,10000,12-11-2025,14-11-2025,15-12-2025 15:00,15-12-2025,0.0,0.000000,NaN
